In [1]:
import pandas as pd

In [4]:
df=pd.read_csv('filtered_scores_debug.csv')
filtered_df=df

In [7]:
cols=[
    'reducto','pymupdf','llamaparse','datalab',
    'gemini','docling','geminipro','llamaparse_highest',
    'datalabllm',
    'pymupdflayout'
]
min_score_threshold=0.25

In [9]:
filtered_df = filtered_df[filtered_df[cols].min(axis=1) < min_score_threshold]

In [10]:
filtered_df.to_csv('filtered_scores_debug1.csv', index=False)

In [2]:
import os
from pathlib import Path
from reducto import Reducto
from abc import ABC, abstractmethod
import os

class PDFConverter(ABC):
    """Abstract base class for PDF to Markdown converters."""

    @abstractmethod
    def convert(self, file_path: str) -> str:
        """
        Convert a PDF file to Markdown.

        Args:
            file_path (str): Path to the PDF file.

        Returns:
            str: The markdown content.
        """
        pass

class ReductoConverter(PDFConverter):
    def __init__(self):
        self.api_key = "e8a149fa9b7d02e4df726bf654a9d10fc1b75f3448c728e8f2eaf98de78c101d6589599c5cff29d6e03514b05c443ab5"
        if not self.api_key:
            raise ValueError("Reductxo API key is required. Set REDUCTO_API_KEY env var or pass it to constructor.")
        
        self.client = Reducto(
            api_key=self.api_key,
            environment="production"
        )
        
        self.options = {
            "ocr_mode": "agentic",
            "chunking": {
                "chunk_mode": "disabled"
            },
            "table_summary": {"enabled": True, "prompt": ""},
            "figure_summary": {"enabled": True, "prompt": "", "override": False}
        }
        
        self.advanced_options = {
            "ocr_system": "multilingual",
            "table_output_format": "md",
            "keep_line_breaks": True,
            "persist_results": True,
            "page_range": {"start": 1, "end": 2} # Keeping original logic, but maybe should be configurable?
        }
        
        self.experimental_options = {}

    def convert(self, file_path: str) -> str:
        print(file_path)
        try:
            # Reducto client expects a file object or path. 
            # The original script used `client.upload(file=pdf_file)` where pdf_file was a Path object.
            path_obj = Path(file_path)
            
            upload_url = self.client.upload(file=path_obj)
            
            result = self.client.parse.run(
                document_url=upload_url,
                options=self.options,
                advanced_options=self.advanced_options,
                experimental_options=self.experimental_options
            )
            
            if result.result.chunks:
                return result.result.chunks[0].content
            return ""
            
        except Exception as e:
            raise RuntimeError(f"Reducto conversion failed: {e}")


In [ ]:
import os
from pathlib import Path
from reducto import Reducto
import sys

# --- Configuration ---
API_KEY = "e8a149fa9b7d02e4df726bf654a9d10fc1b75f3448c728e8f2eaf98de78c101d6589599c5cff29d6e03514b05c443ab5"
if not API_KEY:
    raise ValueError("Reducto API key is required. Set REDUCTO_API_KEY env var or hardcode it.")

OPTIONS = {
    "ocr_mode": "agentic",
    "chunking": {"chunk_mode": "disabled"},
    "table_summary": {"enabled": True, "prompt": ""},
    "figure_summary": {"enabled": True, "prompt": "", "override": False}
}

ADVANCED_OPTIONS = {
    "ocr_system": "multilingual",
    "table_output_format": "md",
    "keep_line_breaks": True,
    "persist_results": True,
    "page_range": {"start": 1, "end": 2}
}

EXPERIMENTAL_OPTIONS = {}

pdf_path = Path("PDFs/page_1.pdf")
if not pdf_path.is_file():
    print(f"Error: File {pdf_path} does not exist.")
    raise FileNotFoundError(f"File {pdf_path} does not exist.")
   

print(f"Converting PDF: {pdf_path}")

# Initialize Reducto client
client = Reducto(api_key=API_KEY, environment="production")

try:
    # Upload PDF
    upload_url = client.upload(file=pdf_path)
    print(f"Uploaded PDF, got URL: {upload_url}")

    # Parse PDF
    result = client.parse.run(
        document_url=upload_url,
        
    )

    # Extract Markdown
    if result.result.chunks:
        markdown = result.result.chunks[0].content
        print("\n--- Markdown Output ---\n")
        print(markdown)
    else:
        print("No content extracted from PDF.")

except Exception as e:
    print(f"Reducto conversion failed: {e}")

Converting PDF: PDFs/page_1.pdf
Uploaded PDF, got URL: Upload(file_id='reducto://5a1389b2-e7e2-467b-b156-1044da527605.pdf', presigned_url=None)
Reducto conversion failed: Missing required argument: 'input'


In [3]:
c=ReductoConverter()
path="PDFs/page_1.pdf"
md=c.convert(path)
print(md)

PDFs/page_1.pdf


RuntimeError: Reducto conversion failed: Missing required argument: 'input'